In [3]:
#This script audits the attachments stored on flywheel, either at the acquisition or session level.
#Inputs:
    #enrollment sheets pulled from axis, stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/
    #data from Flywheel acessed using the CLI 
#Outputs:
    #Binary audit of attachments related to task data (.log, .json, .tsv) on Flywheel 
    #Binary audit of variability on Flywheel 

In [1]:
import flywheel
import pandas as pd
import re 
fw = flywheel.Client()

In [2]:
EFProj=fw.projects.find_first('label=EFR01')
#print(EFProj)

/Users/krmurtha/anaconda3/lib/python3.7/site-packages/flywheel/flywheel.py:6274: UserWarning: Client version 14.6.6 does not match server version 16.4.3. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [3]:
#loop through subs and get sessions
subjects=EFProj.subjects()

sessions=[]
for s in subjects :
    tempsessions=s.sessions()
    sessions.extend(tempsessions)

In [4]:
#read in T1 enrollment
#adjust file path as necessary -- currently assuming parent directory is saturn folder referenced in first block 
axis_t1=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_enroll_t1.csv',dtype=str)
axis_t1=axis_t1.drop(columns=['scan_1_date'])
#and T2
axis_t2=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_enroll_t2.csv',dtype=str)
axis_t2=axis_t2.drop(columns=['scan_2_date'])

In [5]:
#reformat for list of scan IDs 
t1_scan=axis_t1['scan_id_timepoint_1']
t1_scan=t1_scan.tolist()
t1_scan = [str(t) for t in t1_scan]
#and again
t2_scan=axis_t2['scan_id_t2']
t2_scan=t2_scan.tolist()
t2_scan = [str(t) for t in t2_scan]

In [6]:
#get a list of all log files attached at the session level of flywheel 
log_files=[]

for ses in sessions:
    sesid=ses.label
    #print(sesid)
    acq=ses.acquisitions()
    for a in acq:
        if 'func_task-fracnoback_run-02' in a.label or 'ABCD_fMRI_frac-no-back-run1' in a.label:
            EFFiles=a.files
            EFNames=[x.name for x in EFFiles]
            for y in EFNames:
                if 'log' in y:
                    log_files.append((sesid,y))

In [8]:
has_log=[]

for t in t1_scan: 
    if t not in [l[0] for l in log_files]:
        has_log.append((t,0))
    else: 
        has_log.append((t,1))

In [9]:
#save list as a dataframe
t1_audit = pd.DataFrame(has_log, columns =['scanid', 'has_log'])
t1_audit = t1_audit.astype(str)

In [22]:
##now, let's check for various attachments at the session level
import re

# initializing empty lists
has_tsv=[]
has_json=[]
has_variability = []
has_asl=[]

# getting the names of all the files 

EFnames = []
for ii in sessions: # looping through each session
    for jj in range(len(ii["files"])): # looping through each file attached to each session
        file_name = (ii["files"][jj]["name"]) # i.e: accessing the files value within each session's dictionary --> accessing each file in the list of files --> accessing the 'name' value within each dictionary there
        EFnames.append(file_name)

has_json_list = []
for ii in EFnames: # going through each file in the list
    if "bold_events.json" in ii: # all files that have the events.json should also have the events.tsv
        ii = ii.split('_',2) # split so that ses_XXX is isolated as element index 1 in a list
        ii = ii[1] # index ses-*
        has_json_list.append(ii)

has_tsv_list = []
for ii in EFnames: # going through each file in the list
    if "bold_events.tsv" in ii: # all files that have the events.json should also have the events.tsv
        ii = ii.split('_',2) # split so that ses_XXX is isolated as element index 1 in a list
        ii = ii[1] # index ses-*
        has_tsv_list.append(ii)

# repeat for aslcontext and variability 

has_asl_list = []
for ii in EFnames: 
    if "aslcontext" in ii: 
        ii = ii.split('_',2) 
        ii = ii[1] # index ses-*
        has_asl_list.append(ii)
        
has_variability_list = []
for ii in EFnames: 
    if "variability" in ii:
        ii = ii.split('_',2) # variability.zip is formatted differently
        ii = ii[1] 
        has_variability_list.append(ii)

all_sessions_list = []

for t in t1_scan: 
    all_sessions_list.append(t) # append each session number to the all_sessions_list

# creating a list of all sessions, but appending "ses-" at the beginning of each as this is how the information will be pulled out from Flywheel    
all_sessions_list_ses = []  
for ses in all_sessions_list: 
    ses = "ses-" + str(ses)
    all_sessions_list_ses.append(ses)
# print (all_sessions_list_ses)

#looping through, filling out binary audits! 
#check jsons
for item in all_sessions_list_ses: 
    if item not in has_json_list:
        has_json.append((item,0))
    else: 
        has_json.append((item,1))
#check events tsvs        
for item in all_sessions_list_ses: 
    if item not in has_tsv_list:
        has_tsv.append((item,0))
    else: 
        has_tsv.append((item,1))
#check asl context        
for item in all_sessions_list_ses: 
    if item not in has_asl_list :
        has_asl.append((item,0))
    else: 
        has_asl.append((item,1))
#check variability        
for item in all_sessions_list: # variability does not have sub-*-ses-* format, so will not need ses- appended
    if item not in has_variability_list:
        has_variability.append((item,0))
    else: 
        has_variability.append((item,1))
        

In [23]:
#quick reformat to be able to merge! 
has_tsv_reformat=[]
for x in has_tsv:
    y=x[1]
    #z=y.split('-')[0]
    #print(z)
    has_tsv_reformat.append(y)
    
has_json_reformat=[]
for x in has_json:
    y=x[1]
    #z=y.split('-')[0]
    #print(z)
    has_json_reformat.append(y)


In [24]:
#fill in t1 task data audit dataframe 
t1_audit['has_json'] = has_json_reformat
t1_audit['has_tsv'] = has_tsv_reformat

In [25]:
#create an audit data frame for variability
t1_var_audit = pd.DataFrame(has_variability, columns =['scanid', 'has_variability'])

In [26]:
#save sheets to disk 
t1_audit.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/audits/EF_T1_task_audit.csv', sep = ',', index=False)
t1_var_audit.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/audits/EF_T1_variabilty_audit.csv', sep = ',', index=False)

In [27]:
#and check for .log's in T2 scans 
has_log2=[]

for t in t2_scan: 
    if t not in [l[0] for l in log_files]:
        has_log2.append((t,0))
    else: 
        has_log2.append((t,1))

In [28]:
t2_audit = pd.DataFrame(has_log2, columns =['scanid', 'has_log'])

In [29]:
##now, let's check for various attachments at the session level for T2 

# initializing empty lists
has_tsv2=[]
has_json2=[]
has_variability2 = []
has_asl2=[]

all_sessions_list2 = []

for t in t2_scan: 
    all_sessions_list2.append(t) # append each session number to the all_sessions_list

# creating a list of all sessions, but appending "ses-" at the beginning of each as this is how the information will be pulled out from Flywheel    
all_sessions_list_ses2 = []  
for ses in all_sessions_list2: 
    ses = "ses-" + str(ses)
    all_sessions_list_ses2.append(ses)
# print (all_sessions_list_ses)

#looping through, filling out binary audits! 
#check jsons
for item in all_sessions_list_ses2: 
    if item not in has_json_list:
        has_json2.append((item,0))
    else: 
        has_json2.append((item,1))
#check events tsvs        
for item in all_sessions_list_ses2: 
    if item not in has_tsv_list:
        has_tsv2.append((item,0))
    else: 
        has_tsv2.append((item,1))
#check asl context        
for item in all_sessions_list_ses2: 
    if item not in has_asl_list :
        has_asl2.append((item,0))
    else: 
        has_asl2.append((item,1))
#check variability        
for item in all_sessions_list2: # variability does not have sub-*-ses-* format, so will not need ses- appended
    if item not in has_variability_list:
        has_variability2.append((item,0))
    else: 
        has_variability2.append((item,1))
        

In [30]:
#quick reformat to be able to merge! 
has_tsv_reformat2=[]
for x in has_tsv2:
    y=x[1]
    #z=y.split('-')[0]
    #print(z)
    has_tsv_reformat2.append(y)
    
has_json_reformat2=[]
for x in has_json2:
    y=x[1]
    #z=y.split('-')[0]
    #print(z)
    has_json_reformat2.append(y)

In [31]:
t2_audit['has_json']=has_json_reformat2
t2_audit['has_tsv']=has_tsv_reformat2

In [32]:
t2_var_audit = pd.DataFrame(has_variability2, columns =['scanid', 'has_variability'])

In [33]:
t2_audit.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/audits/EF_T2_task_audit.csv', sep = ',', index=False)
t2_var_audit.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/audits/EF_T2_variabilty_audit.csv', sep = ',', index=False)